In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import csv
import time
from collections import Counter
import torch
# import ast

In [4]:
cgm_train = pd.read_csv("Data/cgm_train.csv")
cgm_test = pd.read_csv("Data/cgm_test.csv")
demo_viome_test = pd.read_csv("Data/demo_viome_test.csv")
demo_viome_train = pd.read_csv("Data/demo_viome_train.csv")
img_train = pd.read_csv("Data/img_train.csv")
img_test = pd.read_csv("Data/img_test.csv")
label_train = pd.read_csv("Data/label_train.csv")
label_test_breakfast_only = pd.read_csv("Data/label_test_breakfast_only.csv")

## Data visualization

In [13]:
cgm_train.head()

,Subject ID,Day,Breakfast Time,Lunch Time,CGM Data
0,1,2,2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (..."
1,1,3,2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (..."
2,1,4,2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ..."
3,1,5,2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ..."
4,1,6,2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ..."


In [8]:
demo_viome_train.head()

,Subject ID,Age,Gender,Weight,Height,Race,Diabetes Status,A1C,Baseline Fasting Glucose,Insulin,Triglycerides,Cholesterol,HDL,Non-HDL,LDL,VLDL,CHO/HDL Ratio,HOMA-IR,BMI,Viome
0,1,27,0,133.8,65.00,Hispanic/Latino,1,5.4,91.0,2.5,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."
1,2,49,1,169.2,62.00,Hispanic/Latino,1,5.5,93.0,14.8,61.0,181.0,91.0,90.0,78.0,12.0,2.0,3.398519,30.943704,"-9.395216776716872,-7.315438412832852,1.103724..."
2,3,59,1,157.0,64.00,Hispanic/Latino,3,6.5,118.0,17.4,154.0,190.0,74.0,116.0,90.0,31.0,2.6,5.069630,26.946045,"-11.007947281293239,-8.444134921199856,0.28806..."
3,5,51,1,172.0,62.50,Hispanic/Latino,3,6.6,144.0,12.9,392.0,269.0,38.0,231.0,157.0,78.0,7.1,4.586667,30.954496,"-6.21833166351615,-5.868646092839025,-6.612900..."
4,6,51,1,197.0,68.75,White,1,5.2,96.0,6.4,75.0,203.0,72.0,131.0,118.0,15.0,2.8,1.517037,29.300575,"-6.7953636871125,-6.50799421035929,-1.85678011..."


In [14]:
demo_viome_train.shape, img_train.shape, cgm_train.shape

((36, 20), (324, 6), (324, 5))

In [12]:
img_train.head()

,Subject ID,Day,Breakfast Fiber,Lunch Fiber,Image Before Breakfast,Image Before Lunch
0,1,2,0.0,10,"[[[140, 122, 108], [135, 118, 104], [118, 104,...","[[[41, 152, 201], [77, 164, 205], [88, 157, 13..."
1,1,3,0.0,4,"[[[67, 58, 47], [59, 52, 41], [51, 45, 35], [4...","[[[40, 59, 77], [35, 56, 72], [20, 36, 47], [9..."
2,1,4,0.0,5,"[[[199, 195, 193], [198, 193, 192], [196, 192,...","[[[53, 44, 38], [51, 43, 36], [54, 47, 39], [4..."
3,1,5,7.0,5,"[[[149, 121, 80], [157, 128, 86], [159, 130, 8...","[[[30, 28, 28], [20, 18, 17], [31, 27, 23], [2..."
4,1,6,0.0,18,"[[[175, 184, 198], [192, 206, 219], [160, 165,...","[[[74, 85, 100], [59, 69, 81], [73, 84, 96], [..."


# data visualization

In [ ]:
cgm_train.isnull().sum()

,0
Subject ID,0
Day,0
Breakfast Time,0
Lunch Time,0
CGM Data,0


In [ ]:
demo_viome_train.isnull().sum()

,0
Subject ID,0
Age,0
Gender,0
Weight,0
Height,0
Race,0
Diabetes Status,0
A1C,0
Baseline Fasting Glucose,0
Insulin,0


In [ ]:
img_train.isnull().sum()

,0
Subject ID,0
Day,0
Breakfast Fiber,2
Lunch Fiber,0
Image Before Breakfast,0
Image Before Lunch,0


In [ ]:
img_train.dropna(inplace=True)

In [ ]:
label_train.isnull().sum()

,0
Subject ID,0
Day,0
Breakfast Calories,0
Lunch Calories,0
Breakfast Carbs,0
Lunch Carbs,0
Breakfast Fat,0
Lunch Fat,0
Breakfast Protein,0
Lunch Protein,0


In [ ]:
img_train.shape, cgm_train.shape, demo_viome_train.shape, label_train.shape

((322, 6), (324, 5), (36, 20), (324, 10))

In [ ]:
df = pd.DataFrame(cgm_train)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MultimodalDataset(Dataset):
    def __init__(self, text_data, image_data, numerical_data, labels):
        """
        Args:
            text_data (list or tensor): Text modality data.
            image_data (list or tensor): Image modality data.
            numerical_data (list or tensor): Numerical modality data.
            labels (list or tensor): Labels for each sample.
        """
        self.text_data = text_data
        self.image_data = image_data
        self.numerical_data = numerical_data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = self.image_data[idx]
        numerical = self.numerical_data[idx]
        label = self.labels[idx]

        return {"text": text, "image": image, "numerical": numerical, "label": label}


In [ ]:
# Example data
text_data = torch.randn(100, 300)  # 100 samples, 300-dimensional text embeddings
image_data = torch.randn(100, 3, 224, 224)  # 100 samples, 3x224x224 images
numerical_data = torch.randn(100, 10)  # 100 samples, 10 numerical features
labels = torch.randint(0, 2, (100,))  # Binary labels (0 or 1) for 100 samples

In [ ]:
dataset = MultimodalDataset(text_data, image_data, numerical_data, labels)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
for batch in dataloader:
    text = batch["text"]         # Text embeddings (batch_size, 300)
    image = batch["image"]       # Image tensors (batch_size, 3, 224, 224)
    numerical = batch["numerical"]  # Numerical features (batch_size, 10)
    label = batch["label"]       # Labels (batch_size,)

    print("Text shape:", text.shape)
    print("Image shape:", image.shape)
    print("Numerical shape:", numerical.shape)
    print("Label shape:", label.shape)
    break

Text shape: torch.Size([16, 300])
Image shape: torch.Size([16, 3, 224, 224])
Numerical shape: torch.Size([16, 10])
Label shape: torch.Size([16])
